In [13]:
import pandas
import folium

In [69]:
cocorahs_daily = pandas.read_csv("https://data.cocorahs.org/export/exportreports.aspx?ReportType=Daily&dtf=1&Format=CSV&State=NY&ReportDateType=reportdate&StartDate=9/1/2021&EndDate=9/4/2021&TimesInGMT=False")

In [40]:
cocorahs_sigwx = pandas.read_csv("https://data.cocorahs.org/export/exportreports.aspx?ReportType=SigWx&dtf=1&Format=CSV&State=NY&ReportDateType=reportdate&StartDate=9/1/2021&EndDate=9/4/2021&TimesInGMT=False")

In [70]:
print(f'shape of data: {cocorahs_daily.shape}')
cocorahs_daily.head()

shape of data: (1452, 13)


,ObservationDate,ObservationTime,EntryDateTime,StationNumber,StationName,Latitude,Longitude,TotalPrecipAmt,NewSnowDepth,NewSnowSWE,TotalSnowDepth,TotalSnowSWE,DateTimeStamp
0,2021-09-04,07:30 AM,2021-09-09 07:26 AM,NY-UL-20,Ulster Park 0.9 NNW,41.867677,-73.987955,0.00,0.0,NA,NA,NA,2021-09-09 11:26 AM
1,2021-09-04,07:00 AM,2021-09-04 08:05 AM,NY-OG-61,Skaneateles 0.8 NNW,42.955380,-76.433621,0.01,NA,NA,NA,NA,2021-09-04 12:04 PM
2,2021-09-04,08:00 AM,2021-09-05 10:43 AM,NY-SF-61,Centerport 0.9 SW,40.891689,-73.383133,0.00,0.0,NA,NA,NA,2021-09-05 02:42 PM
3,2021-09-04,07:00 AM,2021-09-04 06:33 PM,NY-GN-20,Alden 2.5 SE,42.874609,-78.458043,0.00,0.0,NA,NA,NA,2021-09-04 10:32 PM
4,2021-09-04,08:00 AM,2021-09-04 02:55 PM,NY-ER-208,Angola 0.6 NNE,42.646111,-79.027500,0.00,0.0,NA,NA,NA,2021-09-04 06:54 PM


In [71]:
cocorahs_daily["Latitude"] = pandas.to_numeric(cocorahs_daily["Latitude"], errors="coerce")
cocorahs_daily["Longitude"] = pandas.to_numeric(cocorahs_daily["Longitude"], errors="coerce")

cocorahs_daily = cocorahs_daily.dropna(subset=["Latitude", "Longitude"])

In [77]:
m = folium.Map(location=[cocorahs_daily["Latitude"].mean(), cocorahs_daily["Longitude"].mean()], zoom_start=7)
folium.TileLayer('cartodbpositron').add_to(m)

for station in cocorahs_daily.iterrows():
    station_values=station[1]
    location=[station_values["Latitude"], station_values["Longitude"]]
    popup=popup=station_values["StationName"]
    marker=folium.CircleMarker(location=location, 
                               popup=popup,
                              radius=3,
                              color="green",
                              fill=True,
                              fill_color="green")
    marker.add_to(m)

m